In [2]:
import seaborn as sns
sns.set_palette('husl')
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import hopsworks
import pandas as pd

/home/redve/Kurser Master/ID2223 Scalable Machine Learning and Deep Learning/ID2223/Lab1/Task1/threepointten/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/190613
Connected. Call `.close()` to terminate connection gracefully.


In [6]:
combined = pd.read_csv("./DataNew/FullDataFrameCalculated.csv")

In [7]:
#remove any commas or strings from the whole dataframe
combined = combined.replace(',','.', regex=True)
combined = combined.replace(' ','', regex=True)
combined = combined.replace('m²','', regex=True)
combined = combined.replace('kr','', regex=True)
combined = combined.replace('rum','', regex=True)
combined = combined.replace('ha','', regex=True)
combined = combined.replace('år','', regex=True)
combined = combined.replace('av','', regex=True)
combined = combined.replace('hiss finns ej','', regex=True)
combined = combined.replace('hiss finns','', regex=True)
combined = combined.replace('hiss fins ej','', regex=True)
combined = combined.replace('hiss fins','', regex=True)
combined = combined.replace('Nej','', regex=True)
combined = combined.replace('Ja','', regex=True)

#change all strings to zeros
combined = combined.replace('',0, regex=True)


#remove everything that is string
combined = combined.apply(pd.to_numeric, errors='coerce')
#filling all the nan values with zeros
combined = combined.fillna(0)



In [8]:
combined.head()

,typeProperty,typeOfForm,rooms,size_x100,bisize_x100,balkong,tomtarea_x10000,buildArea,uteplats,floor,area,type,distance_from_city_center,price_evolution_numbers_per_million,price_evolution_percentage_comma,asking_price_per_million,cost_per_year_x1000,sold_price_per_million,price_per_m2_x10000,elevator
0,1,5,3.0,0.52,0.08,1,0.213,1.964,0,0.0,0.0,1,18.157043,-0.245,-0.08,2.995,14.045,2.75,4.991667,0
1,1,5,3.0,0.60,0.00,1,0.339,1.952,0,0.0,0.0,1,15.884607,-0.195,-0.12,1.695,31.000,1.50,2.825000,0
2,9,5,3.0,0.84,0.00,1,0.209,1.966,0,0.0,0.0,1,13.380714,-0.300,-0.20,1.500,23.700,1.20,1.785714,0
3,1,5,2.0,0.35,0.00,1,0.169,1.958,0,0.0,0.0,1,7.561757,-0.330,-0.23,1.450,6.900,1.12,4.142857,0
4,9,5,2.0,0.45,0.07,1,0.092,1.910,0,0.0,0.0,1,12.192311,-0.095,-0.04,2.395,32.034,2.30,4.605769,0


In [9]:
combined.columns = map(str.lower, combined.columns)
combined.head()

,typeproperty,typeofform,rooms,size_x100,bisize_x100,balkong,tomtarea_x10000,buildarea,uteplats,floor,area,type,distance_from_city_center,price_evolution_numbers_per_million,price_evolution_percentage_comma,asking_price_per_million,cost_per_year_x1000,sold_price_per_million,price_per_m2_x10000,elevator
0,1,5,3.0,0.52,0.08,1,0.213,1.964,0,0.0,0.0,1,18.157043,-0.245,-0.08,2.995,14.045,2.75,4.991667,0
1,1,5,3.0,0.60,0.00,1,0.339,1.952,0,0.0,0.0,1,15.884607,-0.195,-0.12,1.695,31.000,1.50,2.825000,0
2,9,5,3.0,0.84,0.00,1,0.209,1.966,0,0.0,0.0,1,13.380714,-0.300,-0.20,1.500,23.700,1.20,1.785714,0
3,1,5,2.0,0.35,0.00,1,0.169,1.958,0,0.0,0.0,1,7.561757,-0.330,-0.23,1.450,6.900,1.12,4.142857,0
4,9,5,2.0,0.45,0.07,1,0.092,1.910,0,0.0,0.0,1,12.192311,-0.095,-0.04,2.395,32.034,2.30,4.605769,0


In [11]:
import joblib
#normalization or standardization
from sklearn.preprocessing import StandardScaler
y = combined['sold_price_per_million']
X = combined.drop(['sold_price_per_million'], axis=1)
X = X.drop(['price_evolution_numbers_per_million'], axis=1)
X = X.drop(['price_evolution_percentage_comma'], axis=1)
X.head()

scaler = StandardScaler()
scaler.fit(X)
# #save the scaler
scaled_features_test = scaler.transform(X)
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [13]:
# #load the scaler
scaler = joblib.load('scaler.pkl')
scaled_features_test = scaler.transform(X)


In [14]:
from sklearn.decomposition import PCA
pca = PCA(.95)
pca.fit(scaled_features_test)
#save the pca
joblib.dump(pca, 'pca.pkl')


['pca.pkl']

In [17]:
scaler1 = joblib.load('scaler.pkl')

In [18]:
scaled_features_test = scaler1.transform(X)
scaled_features_test = pd.DataFrame(scaled_features_test, columns=X.columns)
scaled_features_test.head()

,typeproperty,typeofform,rooms,size_x100,bisize_x100,balkong,tomtarea_x10000,buildarea,uteplats,floor,area,type,distance_from_city_center,asking_price_per_million,cost_per_year_x1000,price_per_m2_x10000,elevator
0,-1.652549,0.761123,-0.527423,-0.900908,-0.187189,0.203725,0.105817,0.210214,-0.557329,-0.328584,0.0,-0.447214,-0.276185,-0.514669,-0.504898,0.145989,-0.426571
1,-1.652549,0.761123,-0.527423,-0.731912,-0.482275,0.203725,0.272829,0.182325,-0.557329,-0.328584,0.0,-0.447214,-0.368376,-0.908394,0.346843,-0.724048,-0.426571
2,1.263156,0.761123,-0.527423,-0.224923,-0.482275,0.203725,0.100515,0.214862,-0.557329,-0.328584,0.0,-0.447214,-0.469956,-0.967452,-0.019875,-1.141379,-0.426571
3,-1.652549,0.761123,-1.075566,-1.260025,-0.482275,0.203725,0.047495,0.196270,-0.557329,-0.328584,0.0,-0.447214,-0.706026,-0.982596,-0.863830,-0.194855,-0.426571
4,1.263156,0.761123,-1.075566,-1.048779,-0.224074,0.203725,-0.054567,0.084714,-0.557329,-0.328584,0.0,-0.447214,-0.518169,-0.696388,0.398786,-0.008970,-0.426571


In [19]:
Y = combined['sold_price_per_million']
scaled_features_test['sold_price_per_million'] = Y


In [21]:
bostad = fs.get_or_create_feature_group(name='bostad'
                                           ,version='4'
                                           ,primary_key=['typeproperty','typeofform','rooms','size_x100','bisize_x100','balkong','tomtarea_x10000','buildarea','uteplats','area','type','distance_from_city_center','asking_price_per_million','cost_per_year_x1000','sold_price_per_million','price_per_m2_x10000','elevator']
                                            ,description='Bostad data')
bostad.insert(scaled_features_test)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/190613/fs/190532/fg/395625


Uploading Dataframe: 100.00% |██████████| Rows 20000/20000 | Elapsed Time: 00:13 | Remaining Time: 00:00


Launching job: bostad_4_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/190613/jobs/named/bostad_4_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f40dc6c67d0>, None)